<a href="https://colab.research.google.com/github/atifalimd/ML-Projects/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files
iles.upload()##upload kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"atifalimd","key":"02b78047d307e8639d3f15894b83d04c"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download -d rounakbanik/the-movies-dataset

 97% 221M/228M [00:03<00:00, 49.6MB/s]
100% 228M/228M [00:03<00:00, 67.8MB/s]


In [ ]:
!unzip 'the-movies-dataset.zip'

Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             

In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
movies=pd.read_csv('movies_metadata.csv')

In [ ]:
keywords=pd.read_csv('keywords.csv')

In [ ]:
credits=pd.read_csv('credits.csv')

In [ ]:
movies.head(3)

In [ ]:
movies.columns

In [ ]:
drop_feat=[ 'homepage','poster_path','production_countries','spoken_languages','tagline','status','belongs_to_collection','original_title','overview','video','spoken_languages','production_companies','runtime']

In [ ]:
movies=movies.drop(columns=drop_feat)

In [ ]:
movies

In [ ]:
movies.info()

In [ ]:
drop_rows=[]
for index,row in movies.iterrows():
  row['id']=row['id'].split('-')
  if len(row['id'])>1:
    drop_rows.append(index)
print(drop_rows)

In [ ]:
movies=movies.drop(index=drop_rows)

In [ ]:
movies['id']=movies['id'].astype(int)

In [ ]:
movies=movies.merge(keywords,on='id')

In [ ]:
df=movies.merge(credits,on='id')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df['adult'].value_counts()

In [ ]:
df['adult'][df['adult']=='False']

In [ ]:
fig=plt.figure(figsize=[25,15])
ax=fig.subplots()
df['original_language'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
df['original_language'].value_counts()

In [ ]:
'''we will go with only english language'''

In [ ]:
df=df[df['original_language']=='en']

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
top_15_revenue_movies=df['revenue'].nlargest(15)

In [ ]:
top_15_revenue_movies=top_15_revenue_movies.index

In [ ]:
highest_revenue_movies=df.iloc[top_15_revenue_movies]

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
sns.barplot(x='title',y='revenue',data=highest_revenue_movies)
plt.xticks(rotation=300)
plt.show()

In [ ]:
'Budget,popularity,votes,date'

In [ ]:
df.head(2)

In [ ]:
df['budget'].sort_values(ascending=False)[:15]

In [ ]:
df['budget']=df['budget'].astype(int)

In [ ]:

top_15_budget=df['budget'].nlargest(15)

In [ ]:
top_15_budget_ind=top_15_budget.index

In [ ]:
max_budget_movies=df.iloc[top_15_budget_ind]

In [ ]:
import plotly.express as px

In [ ]:
plt.figure(figsize=(20,15))
fig=px.histogram(max_budget_movies,x='title',y='budget',color="budget",title='max_budget_movies')
labels=dict(title='Movies',budget='Budget')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
df['popularity']=df['popularity'].astype(float)

In [ ]:
top_15_popular=df['popularity'].nlargest(15)

In [ ]:
top_15_popular_ind=top_15_popular.index.to_list()

In [ ]:
most_popular_movies=df.iloc[top_15_popular_ind]

In [ ]:
plt.figure(figsize=(20,15))
fig=px.histogram(data_frame=most_popular_movies,x='title',y='popularity',color='popularity',title='Most Popular Movies')
labels=dict(title='Movies',popularity='Popularity')
fig.update_layout(barmode='stack',xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
df['vote_count']=df['vote_count'].astype(int)

In [ ]:
top_15_vote=df['vote_count'].nlargest(15).index

In [ ]:
max_vote_movies=df.iloc[top_15_vote]

In [ ]:
plt.figure(figsize=(20,15))
fig=px.histogram(data_frame=max_vote_movies,x='title',y='vote_count',color='vote_count',title='max_vote_movies')
labels=dict(title='Movies',vote_count='Vote Count')
fig.update_layout(barmode='stack',xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
df['release_year']=df['release_date'].apply(lambda x:x.split('-')[0])

In [ ]:
df['release_year']=df['release_year'].astype(int)

In [ ]:
df.drop(columns=['release_date'],inplace=True)

In [ ]:
df.drop('vote_average',axis=1,inplace=True)

In [ ]:
(df['release_year']<1990).shape

In [ ]:
drop_rows=df[df['release_year']<1990].index

In [ ]:
df.drop(index=drop_rows,inplace=True)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
data=df.copy()

In [ ]:
data.drop(columns=['adult','budget','popularity','revenue','vote_count','imdb_id','original_language'],inplace=True)

In [ ]:
data.sample(3)

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
def strng_collection(text):
  text=re.sub("[^a-zA-Z]"," ",text) ###re.sub("[a-zA-Z]","",text): Means substitute anything  except a-zA-Z with blank space
  text=text.lower()
  text=text.replace("id","")
  text=text.replace('name','')
  text=nltk.word_tokenize(text)
  text=' '.join(text)
  return text

In [ ]:
data['genres']=data['genres'].apply(strng_collection)

In [ ]:
data['keywords']=data['keywords'].apply(strng_collection)

In [ ]:
data.head()

In [ ]:
import ast

In [ ]:
def convert_to_word(text):
  l=[]
  for i in ast.literal_eval(text):
    l.append(i['name'].lower())
  return l

In [ ]:
data['crew']=data['crew'].apply(convert_to_word)

In [ ]:
data['cast']=data['cast'].apply(convert_to_word)

In [ ]:
data.head(3)

In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
data['genres']=data['genres'].apply(lambda x:x.split())

In [ ]:
data['title']=data['title'].apply(lambda x:x.split())

In [ ]:
data['keywords']=data['keywords'].apply(lambda x:x.split())

In [ ]:
data.head()

In [ ]:
obj_cols=data.select_dtypes('object').columns

In [ ]:
l=[]
for i in obj_cols:
    l.append(data[data[i].apply(lambda x:len(x)==0)].index.to_list())

In [ ]:
l1,l2,l3=l[2:]

In [ ]:
drop_indices=l1+l2+l3

In [ ]:
data.drop(index=drop_indices,inplace=True)

In [ ]:
data

In [ ]:
for i in obj_cols:
  data[i]=data[i].apply(lambda x: ' '.join(x))

In [ ]:
data

In [ ]:
from nltk import PorterStemmer,WordNetLemmatizer

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stemmer=PorterStemmer()
def stem(text):
  text=text.split(" ")
  text=[stemmer.stem(word) for word in text if word not in stopwords.words('english')]
  text=' '.join(text)
  return text

In [ ]:
for index, row in data.iterrows() :
  text = row["keywords"]
  preprocessed_text = stem(text)
  row["keywords_data"] = preprocessed_text

In [ ]:
data.head()

In [ ]:
data["keywords"][1056]

In [ ]:
credits=pd.read_csv('credits.csv')

In [ ]:
credits

In [ ]:
def whole_word(text):
  text=ast.literal_eval(text)
  return text

In [ ]:
credits['crew']=credits['crew'].apply(whole_word)

In [ ]:
credits.reset_index(inplace=True)

In [ ]:
credits.head()

In [ ]:
crew_len=len(credits['crew'])

In [ ]:
crew_len

In [ ]:
crew=pd.Series(credits['crew'])

In [109]:
crew

0        [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1        [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2        [{'credit_id': '52fe466a9251416c75077a89', 'de...
3        [{'credit_id': '52fe44779251416c91011acb', 'de...
4        [{'credit_id': '52fe44959251416c75039ed7', 'de...
                               ...                        
45471    [{'credit_id': '5894a97d925141426c00818c', 'de...
45472    [{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...
45473    [{'credit_id': '52fe4776c3a368484e0c8387', 'de...
45474    [{'credit_id': '533bccebc3a36844cf0011a7', 'de...
45475    [{'credit_id': '593e676c92514105b702e68e', 'de...
Name: crew, Length: 45476, dtype: object

In [110]:
l_name=[]
l_i=[]
for i in range(crew_len):
  for j in crew[i]:
    if j['job']=='Director':
      l_name.append(j['name'])
      l_i.append(i)

In [111]:
temp=pd.DataFrame(l_name,l_i,columns=['Director'])

In [112]:
temp

,Director
0,John Lasseter
1,Joe Johnston
2,Howard Deutch
3,Forest Whitaker
4,Charles Shyer
...,...
45471,Hamid Nematollah
45472,Lav Diaz
45473,Mark L. Lester
45474,Yakov Protazanov


In [113]:
temp.reset_index(inplace=True)

In [114]:
temp.shape

(49048, 2)

In [115]:
index=temp.index.tolist()

In [116]:
temp.head()

,index,Director
0,0,John Lasseter
1,1,Joe Johnston
2,2,Howard Deutch
3,3,Forest Whitaker
4,4,Charles Shyer


In [117]:
credits.shape

(45476, 4)

In [118]:
credits=pd.merge(credits,temp,how='right',on='index')

In [119]:
data.head()

,genres,id,title,keywords,cast,crew,release_year
0,animation comedy family,862,Toy Story,jealousy toy boy friendship friends rivalry bo...,tom hanks tim allen don rickles jim varney wal...,john lasseter joss whedon andrew stanton joel ...,1995
1,adventure fantasy family,8844,Jumanji,board game disappearance based on children s b...,robin williams jonathan hyde kirsten dunst bra...,larry j. franco jonathan hensleigh james horne...,1995
2,romance comedy,15602,Grumpier Old Men,fishing best friend duringcreditsstinger old men,walter matthau jack lemmon ann-margret sophia ...,howard deutch mark steven johnson mark steven ...,1995
3,comedy drama romance,31357,Waiting to Exhale,based on novel interracial relationship single...,whitney houston angela bassett loretta devine ...,forest whitaker ronald bass ronald bass ezra s...,1995
4,comedy,11862,Father of the Bride Part II,baby mlife crisis confence aging daughter moth...,steve martin diane keaton martin short kimberl...,alan silvestri elliot davis nancy meyers nancy...,1995


In [120]:
credits.head()

,index,cast,crew,id,Director
0,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [121]:
credits.drop_duplicates(inplace=True)

TypeError: ignored

In [122]:
credits.shape

(49048, 5)

In [124]:
df=data.merge(credits,on='id')

In [125]:
df=df[['cast_x','keywords','Director','genres','id','release_year','title']]

In [126]:
df.head()

,cast_x,keywords,Director,genres,id,release_year,title
0,tom hanks tim allen don rickles jim varney wal...,jealousy toy boy friendship friends rivalry bo...,John Lasseter,animation comedy family,862,1995,Toy Story
1,robin williams jonathan hyde kirsten dunst bra...,board game disappearance based on children s b...,Joe Johnston,adventure fantasy family,8844,1995,Jumanji
2,walter matthau jack lemmon ann-margret sophia ...,fishing best friend duringcreditsstinger old men,Howard Deutch,romance comedy,15602,1995,Grumpier Old Men
3,whitney houston angela bassett loretta devine ...,based on novel interracial relationship single...,Forest Whitaker,comedy drama romance,31357,1995,Waiting to Exhale
4,steve martin diane keaton martin short kimberl...,baby mlife crisis confence aging daughter moth...,Charles Shyer,comedy,11862,1995,Father of the Bride Part II


In [127]:
df['tag']=df['cast_x']+df['keywords']+df['genres']+df['Director']

In [128]:
df=df[['id','title','release_year','tag']]

In [129]:
df.shape

(15301, 4)

In [130]:
df.isnull().sum()

id              0
title           0
release_year    0
tag             0
dtype: int64

In [131]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy.sparse import csr_matrix

In [132]:
cv=CountVectorizer()
cv.fit(df['tag'])

CountVectorizer()

In [133]:
x=cv.transform(df['tag'])

In [134]:
import scipy
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [135]:
df1=pd.DataFrame.sparse.from_spmatrix(x)

In [136]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15301 entries, 0 to 15300
Columns: 87422 entries, 0 to 87421
dtypes: Sparse[int64, 0](87422)
memory usage: 7.1 MB


In [137]:
max(df1.max())

9

In [138]:
#since the max value in dataframe is 9 hence we can change data type from int64 to int8

In [139]:
df1=df1.astype('int8')

In [140]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15301 entries, 0 to 15300
Columns: 87422 entries, 0 to 87421
dtypes: Sparse[int8, 0](87422)
memory usage: 2.9 MB


In [141]:
#converting sparse dataframe into spare matrix
sdf = df1.astype(pd.SparseDtype("int8", 0))
sdf_matrix = scipy.sparse.csr_matrix(sdf.values)

<15301x87422 sparse matrix of type '<class 'numpy.int8'>'
	with 617868 stored elements in Compressed Sparse Row format>

In [145]:
df['title']=df['title'].apply(lambda x:x.lower())

In [146]:
print(sdf_matrix)

  (0, 2021)	1
  (0, 3069)	1
  (0, 9427)	2
  (0, 15259)	1
  (0, 15692)	1
  (0, 19845)	1
  (0, 21788)	1
  (0, 21928)	1
  (0, 25033)	1
  (0, 25057)	1
  (0, 26152)	1
  (0, 29301)	1
  (0, 29445)	1
  (0, 29458)	1
  (0, 34111)	1
  (0, 40017)	1
  (0, 40027)	1
  (0, 40197)	2
  (0, 45399)	1
  (0, 45534)	1
  (0, 45909)	1
  (0, 46741)	1
  (0, 52555)	1
  (0, 54336)	1
  (0, 56874)	1
  :	:
  (15299, 84760)	1
  (15299, 85147)	1
  (15299, 85782)	1
  (15300, 5562)	1
  (15300, 7757)	1
  (15300, 11144)	1
  (15300, 18634)	1
  (15300, 21796)	1
  (15300, 22911)	1
  (15300, 29154)	1
  (15300, 32409)	1
  (15300, 34914)	1
  (15300, 40495)	1
  (15300, 42702)	1
  (15300, 46019)	1
  (15300, 46269)	1
  (15300, 47991)	1
  (15300, 52680)	1
  (15300, 53642)	1
  (15300, 54070)	1
  (15300, 55876)	1
  (15300, 60018)	1
  (15300, 66134)	1
  (15300, 70604)	1
  (15300, 84841)	1


In [147]:
cos_similar=cosine_similarity(sdf_matrix)

In [149]:
pd.DataFrame(cos_similar)

,0,1,2,3,4,5,6,7,8,9,...,15291,15292,15293,15294,15295,15296,15297,15298,15299,15300
0,1.000000,0.032530,0.0,0.000000,0.000000,0.029311,0.045361,0.0,0.046524,0.112154,...,0.077152,0.000000,0.0,0.000000,0.027217,0.065372,0.077152,0.077152,0.042505,0.000000
1,0.032530,1.000000,0.0,0.021129,0.054681,0.017162,0.029881,0.0,0.027242,0.032835,...,0.000000,0.000000,0.0,0.024398,0.000000,0.057417,0.045175,0.045175,0.037333,0.050965
2,0.000000,0.000000,1.0,0.000000,0.000000,0.015667,0.018185,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.021129,0.0,1.000000,0.080875,0.063459,0.044194,0.0,0.040291,0.072846,...,0.033408,0.023837,0.0,0.000000,0.035355,0.000000,0.033408,0.033408,0.000000,0.037689
4,0.000000,0.054681,0.0,0.080875,1.000000,0.076641,0.063541,0.0,0.017379,0.083789,...,0.028820,0.020563,0.0,0.031129,0.030500,0.024419,0.057639,0.057639,0.023816,0.032513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15296,0.065372,0.057417,0.0,0.000000,0.024419,0.011497,0.026688,0.0,0.054745,0.000000,...,0.030261,0.000000,0.0,0.000000,0.032026,1.000000,0.000000,0.000000,0.000000,0.000000
15297,0.077152,0.045175,0.0,0.033408,0.057639,0.013568,0.094491,0.0,0.043073,0.129794,...,0.035714,0.025482,0.0,0.000000,0.037796,0.000000,1.000000,0.928571,0.029514,0.080582
15298,0.077152,0.045175,0.0,0.033408,0.057639,0.013568,0.094491,0.0,0.043073,0.129794,...,0.035714,0.025482,0.0,0.000000,0.037796,0.000000,0.928571,1.000000,0.029514,0.080582
15299,0.042505,0.037333,0.0,0.000000,0.023816,0.056063,0.039043,0.0,0.017798,0.064356,...,0.000000,0.042117,0.0,0.000000,0.031235,0.000000,0.029514,0.029514,1.000000,0.099889


In [197]:
title=pd.Series(df['title'])

In [198]:
title.head()

0                      toy story
1                        jumanji
2               grumpier old men
3              waiting to exhale
4    father of the bride part ii
Name: title, dtype: object

In [199]:
title[title=='thor'].index[0]

6838

In [260]:
while True:
  movie=input('Enter the movie name: ').lower()
  def recommendation(movie):
    movie_id=title[title==movie].index[0]
    score=pd.Series(cos_similar[movie_id]).sort_values(ascending=False)
    top_10_movie_id=score[1:11].index.to_list()
    similar_10_movies=df.iloc[top_10_movie_id][['title','release_year']].reset_index(drop=True)
    #similar_10_movies=pd.DataFrame(similar_10_movies)
    similar_10_movies = similar_10_movies.set_index(pd.Index([1,2,3,4,5,6,7,8,9,10]))
    return similar_10_movies
  try:
      print(recommendation(movie))
      break
  except:
      print('Please enter the name correctly')

Enter the movie name: spider-man
                         title  release_year
1                 spider-man 2          2004
2                 spider-man 3          2007
3   oz: the great and powerful          2013
4              drag me to hell          2009
5                   iron man 2          2010
6                        hoffa          1992
7               run ronnie run          2002
8                   black mass          2015
9                       2 guns          2013
10                  deterrence          2000
